In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import math
from datetime import datetime,timedelta
from sklearn import preprocessing

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 6, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False  

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Data Preprocess

In [3]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

train.rename(columns={'Unnamed: 0' : 'id'}, inplace=True)
test.rename(columns={'Unnamed: 0' : 'id'}, inplace=True)

train['id'] = -1
train.head()

In [ ]:
test['target'] = -1
train.shape
test.shape

full_data = pd.concat([train, test])
full_data.shape

full_data.head()

del train
del test

In [ ]:
songs = pd.read_csv('./input/songs.csv')
members = pd.read_csv('./input/members.csv')

songs = songs.drop(['Unnamed: 0'], axis=1)
members = members.drop(['Unnamed: 0'], axis=1)

songs.head(5)
members.head(5)

In [ ]:
full_data = full_data.merge(songs, left_on='song_id', right_on='song_id', how='left')
full_data = full_data.merge(members, left_on='msno', right_on='msno', how='left')

del songs
del members

full_data.shape
full_data.head(6)

In [6]:
full_data.isnull().sum()[full_data.isnull().sum() != 0]

song_length       2365
genre_ids       162772
artist_name       2365
composer       2296450
lyricist       4404583
language          2365
name              2365
isrc            774628
dtype: int64

In [7]:
full_data = full_data.drop(['isrc'], axis=1)
full_data = full_data.drop(['song_length'], axis=1)

full_data.genre_ids[full_data.genre_ids.isnull() == True] = 'None'
full_data.composer[full_data.composer.isnull() == True] = 'None'
full_data.lyricist[full_data.lyricist.isnull() == True] = 'None'
full_data.artist_name[full_data.artist_name.isnull() == True] = 'None'
full_data.name[full_data.name.isnull() == True] = 'None'

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/miniconda2/envs/py35/lib/python3.5/site-pac

In [8]:
full_data.registration_init_time = pd.to_datetime(full_data.registration_init_time, format="%Y-%m-%d")
full_data.expiration_date = pd.to_datetime(full_data.expiration_date, format="%Y-%m-%d")
full_data.registration_days = full_data.expiration_date - full_data.registration_init_time

In [9]:
full_data['registration_last_days'] = full_data.registration_days.apply(lambda x : x.days).astype(int)
full_data['registration_year'] = full_data.registration_init_time.apply(lambda x : x.year).astype(int)
full_data['registration_month'] = full_data.registration_init_time.apply(lambda x : x.month).astype(int)
full_data['registration_day'] = full_data.registration_init_time.apply(lambda x : x.day).astype(int)

full_data = full_data.drop(['registration_init_time', 'expiration_date', 'registration_days'], axis=1)

In [10]:
split_fa = lambda x : x.split('|')
full_data.genre_ids = full_data.genre_ids.apply(split_fa)

def spl_col(x):
    if x != []:
        return x.pop()
    else:
        return x

full_data['genre_ids1'] = full_data.genre_ids.apply(spl_col)
full_data['genre_ids2'] = full_data.genre_ids.apply(spl_col)
full_data['genre_ids3'] = full_data.genre_ids.apply(spl_col)
full_data['genre_ids4'] = full_data.genre_ids.apply(spl_col)
full_data['genre_ids5'] = full_data.genre_ids.apply(spl_col)
full_data['genre_ids6'] = full_data.genre_ids.apply(spl_col)
full_data['genre_ids7'] = full_data.genre_ids.apply(spl_col)
full_data['genre_ids8'] = full_data.genre_ids.apply(spl_col)

full_data = full_data.drop(['genre_ids'], axis=1)

In [47]:
full_data.language[full_data.language.apply(math.isnan)] = 250
full_data.language = full_data.language.astype(int)
full_data.gender = full_data.gender.astype(int)

def int_genre(x):
    if type(x) == str:
        if x == 'None':
            return 0
        else:
            return int(x)
    elif type(x) == int:
        return x
    else:
        if x != []:
            return int(x.pop())
        else:
            return 0

full_data.genre_ids1 = full_data.genre_ids1.apply(int_genre)
full_data.genre_ids2 = full_data.genre_ids2.apply(int_genre)
full_data.genre_ids3 = full_data.genre_ids3.apply(int_genre)
full_data.genre_ids4 = full_data.genre_ids4.apply(int_genre)
full_data.genre_ids5 = full_data.genre_ids5.apply(int_genre)
full_data.genre_ids6 = full_data.genre_ids6.apply(int_genre)
full_data.genre_ids7 = full_data.genre_ids7.apply(int_genre)
full_data.genre_ids8 = full_data.genre_ids8.apply(int_genre)

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
full_data.to_csv('./input/full_data_mode.csv')

In [49]:
msno_le = preprocessing.LabelEncoder().fit(full_data['msno'])
full_data.msno = msno_le.transform(full_data.msno)

composer_le = preprocessing.LabelEncoder().fit(full_data['composer'])
full_data['composer'] = composer_le.transform(full_data['composer'])

lyricist_le = preprocessing.LabelEncoder().fit(full_data['lyricist'])
full_data['lyricist'] = lyricist_le.transform(full_data['lyricist'])

song_id_le = preprocessing.LabelEncoder().fit(full_data['song_id'])
full_data['song_id'] = song_id_le.transform(full_data['song_id'])

artist_name_le = preprocessing.LabelEncoder().fit(full_data['artist_name'])
full_data['artist_name'] = artist_name_le.transform(full_data['artist_name'])

name_le = preprocessing.LabelEncoder().fit(full_data['name'])
full_data['name'] = name_le.transform(full_data['name'])

In [50]:
full_data.shape
full_data.head()

(9934208, 68)

,id,msno,name_Album more,name_Artist more,name_Concert,name_Discover Chart,name_Discover Feature,name_Discover Genre,name_Discover New,name_Explore,...,registration_month,registration_day,genre_ids1,genre_ids2,genre_ids3,genre_ids4,genre_ids5,genre_ids6,genre_ids7,genre_ids8
0,-1,9176,0,0,0,0,0,0,0,1,...,1,2,359,0,0,0,0,0,0,0
1,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1259,0,0,0,0,0,0,0
2,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1259,0,0,0,0,0,0,0
3,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1019,0,0,0,0,0,0,0
4,-1,9176,0,0,0,0,0,0,0,1,...,1,2,1011,0,0,0,0,0,0,0


In [52]:
full_data.to_csv('./input/full_data_mode_encode.csv')

In [ ]:
# for index, row in full_data.groupby(['msno']): 
#     for x in row:
#         genre_x = str(x.genre_ids1) + ' ' + str(x.genre_ids2)
#         + ' ' + str(x.genre_ids3) + ' ' + str(x.genre_ids4) 
#         + ' ' + str(x.genre_ids5) + ' ' + str(x.genre_ids6) 
#         + ' ' + str(x.genre_ids7) + ' ' + str(x.genre_ids8)

## Train

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.model_selection import GridSearchCV

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
full_data = pd.read_csv('./input/full_data_mode_encode.csv')
full_data = full_data.drop(['Unnamed: 0'], axis=1)

full_data.shape
full_data.head()

# for i in full_data.columns:
#     if (i != 'bd') & (i != 'registration_last_days'):
#         full_data[i] = full_data[i].astype('category')

train = full_data[full_data.id == -1]
test = full_data[full_data.id != -1]

ids = test.id
train = train.drop(['id'], axis=1)
test = test.drop(['id', 'target'], axis=1)

del full_data

(9934208, 68)

,id,msno,name_Album more,name_Artist more,name_Concert,name_Discover Chart,name_Discover Feature,name_Discover Genre,name_Discover New,name_Explore,...,registration_month,registration_day,genre_ids1,genre_ids2,genre_ids3,genre_ids4,genre_ids5,genre_ids6,genre_ids7,genre_ids8
0,-1,9176,0,0,0,0,0,0,0,1,...,1,2,359,0,0,0,0,0,0,0
1,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1259,0,0,0,0,0,0,0
2,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1259,0,0,0,0,0,0,0
3,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1019,0,0,0,0,0,0,0
4,-1,9176,0,0,0,0,0,0,0,1,...,1,2,1011,0,0,0,0,0,0,0


In [4]:
x_train = train.drop(['target'], axis=1)
y_train = train.target
x_test = test

del train
del test

x_train.shape
x_test.shape
x_train.head()

(7377418, 66)

(2556790, 66)

,msno,name_Album more,name_Artist more,name_Concert,name_Discover Chart,name_Discover Feature,name_Discover Genre,name_Discover New,name_Explore,name_Local playlist more,...,registration_month,registration_day,genre_ids1,genre_ids2,genre_ids3,genre_ids4,genre_ids5,genre_ids6,genre_ids7,genre_ids8
0,9176,0,0,0,0,0,0,0,1,0,...,1,2,359,0,0,0,0,0,0,0
1,19273,0,0,0,0,0,0,0,0,1,...,5,25,1259,0,0,0,0,0,0,0
2,19273,0,0,0,0,0,0,0,0,1,...,5,25,1259,0,0,0,0,0,0,0
3,19273,0,0,0,0,0,0,0,0,1,...,5,25,1019,0,0,0,0,0,0,0
4,9176,0,0,0,0,0,0,0,1,0,...,1,2,1011,0,0,0,0,0,0,0


In [8]:
from sklearn.decomposition import PCA

In [9]:
pca = PCA(n_components='mle', svd_solver='auto', copy=True)

pca.fit(x_train)
pca.explain_variance_ratio_
x_train_pca = pca.transform(x_train)
x_train_pca.head()

TypeError: unorderable types: str() >= int()

In [5]:
rf_model = RandomForestClassifier(
    n_estimators=200, 
    max_depth=6, 
    min_samples_leaf=15,
    n_jobs=-1,
)

rf_model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=15,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [6]:
y_test = rf_model.predict_proba(x_test)
y_train_pred = rf_model.predict_proba(x_train)

In [7]:
sub = pd.DataFrame()
sub['id'] = ids
sub['target'] = y_test[:, 1]
sub
sub.to_csv('./submit/rf_2.csv', index=False, float_format = '%.6f')

,id,target
7377418,0,0.635496
7377419,1,0.638201
7377420,2,0.373883
7377421,3,0.231678
7377422,4,0.230307
7377423,5,0.229155
7377424,6,0.230041
7377425,7,0.637201
7377426,8,0.245798
7377427,9,0.624783


In [8]:
y_train = pd.DataFrame(y_train_pred[:, 1])
y_train.to_csv('./submit/rf_train.csv', index=False, float_format = '%.6f')